In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
import torch 
from torch import nn
from torch.utils.data import DataLoader, random_split
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
import torch.nn.functional as F
import numpy as np

In [18]:
transform = transforms.Compose([
    transforms.Resize((227,227)),
    transforms.ToTensor(),
#     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
#     transforms.functional.rgb_to_grayscale()
])

batch_size = 128

data_dir = '/kaggle/input/concrete-crack-images-for-classification'

data = datasets.ImageFolder(data_dir, transform=transform)

# dataloader = DataLoader(data,batch_size=batch_size,shuffle=True)

In [19]:
n=len(data)
train_size = int(n*0.8)
test_size= int(n*0.2)
data_train, data_test = random_split(data,[train_size,test_size])

In [20]:
dataloader_train = DataLoader(data_train,batch_size=batch_size,shuffle=True)
dataloader_test = DataLoader(data_test,batch_size=batch_size,shuffle=False)

In [21]:
torch.cuda.is_available()

True

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
import torch
import torchvision
import torch.nn as nn 
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self,num_classes):
        super(AlexNet,self).__init__()
        self.conv1 = nn.Conv2d(3,96,11,stride=4,padding=0)
        self.maxPool = nn.MaxPool2d(kernel_size=3,stride=2)
        self.conv2 = nn.Conv2d(96,256,5,stride=1,padding=2)
        self.conv3 = nn.Conv2d(256,384,3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(384,384,3,stride=1,padding=1)
        self.conv5 = nn.Conv2d(384,256,3,stride=1,padding=1)
        self.fc1 = nn.Linear(256*6*6,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,num_classes)
        self.b1 = nn.BatchNorm2d(96)
        self.b2 = nn.BatchNorm2d(256)
        self.b3 = nn.BatchNorm2d(384)
        self.d = nn.Dropout1d(0.5)

    def forward(self,x):
        x = self.maxPool(F.relu(self.b1(self.conv1(x))))
        x = self.maxPool(F.relu(self.b2(self.conv2(x))))
        x = F.relu(self.b3(self.conv3(x)))
        x = F.relu(self.b3(self.conv4(x)))
        x = self.maxPool(F.relu(self.b2(self.conv5(x))))
        # x = torch.flatten(x,1)
        x = x.reshape(x.size(0),-1)
        x = F.relu(self.fc1(self.d(x)))
        x = F.relu(self.fc2(self.d(x)))
        x = F.relu(self.fc3(x))
        return x


In [24]:
class AlexNetB(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNetB, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [25]:
# from AlexNet import *
modelB = AlexNetB(2).to(device)
model = AlexNet(2).to(device)
model = model
learning_rate = 1e-5
epochs = 1

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for epoch in range(epochs):
    for idx,(img,target) in enumerate(dataloader_train):
        img = img.to(device=device)
        target = target.to(device=device)

        #forward pass
        y_pred = model(img)

        loss = criterion(y_pred,target)

        # backward pass
        loss.backward()

        # weight update
        optimizer.step()

        optimizer.zero_grad()

        

In [26]:
PATH = '/kaggle/working/AlexNet'
torch.save(model.state_dict(), PATH)

In [27]:
# PATHB = '/kaggle/working/AlexNetB'
# torch.save(modelB.state_dict(), PATHB)

In [28]:
model = AlexNet(2)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in dataloader_test:
        images, labels = data
#         labels = labels.to(device=device)
#         images = images.to(device=device)
        # calculate outputs by running images through the network
        outputs = model(images)
#         print(labels,outputs)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {len(dataloader_test)*128} test images: {100 * correct / total} %')

In [ ]:
# prepare to count predictions for each class
classes = ('Negative','Positive')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in dataloader_test:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')